# Setup

https://github.com/radioxoma/micromanager-samples/blob/master/mm_live_video.py

https://micro-manager.org/wiki/Using_the_Micro-Manager_python_library#Using_Python_API

https://micro-manager.org/wiki/Micro-Manager_Programming_Guide#Other


http://micro-manager.3463995.n2.nabble.com/Cannot-add-Andor-SDK3-device-to-configuration-td7583283.html  
https://sourceforge.net/p/micro-manager/mailman/message/34388695/

# Main
https://micro-manager.org/wiki/Micro-Manager_Programming_Guide   
https://valelab4.ucsf.edu/~MM/doc/MMCore/html/class_c_m_m_core.html

In [ ]:
# !!!! Also need to add MM folder to system PATH

# mm_version = 'C:\Micro-Manager-1.4'
# cfg = 'C:\Micro-Manager-1.4\SetupNumber2_05102016.cfg'
mm_version = 'C:\Program Files\Micro-Manager-2.0beta'
cfg = 'C:\Program Files\Micro-Manager-2.0beta\Setup2_20170413.cfg'

import sys
sys.path.insert(0, mm_version) # make it so python can find MMCorePy
import MMCorePy

import time
from PIL import Image
import numpy as np
import matplotlib.pyplot as plt
%matplotlib inline

In [ ]:
core = MMCorePy.CMMCore()
core.loadSystemConfiguration(cfg)

In [ ]:
core.initializeCircularBuffer()
core.setCircularBufferMemoryFootprint(4096) # MiB 

In [ ]:
core.setProperty(core.getCameraDevice(), "Exposure", 300)

In [ ]:
core.setProperty("Spectra", "White_Enable", "1")
core.waitForDevice("Spectra")

In [ ]:
# NEED TO SET CAMERA TO 16 BIT (ceiling 12 BIT = 4096)
core.setProperty("Cam Andor_Zyla4.2", "Sensitivity/DynamicRange", "16-bit (low noise & high well capacity)")

Preset: 1_PBP   
ConfigGroup,Channel,1_PBP,TIFilterBlock1,Label,1-PBP

Preset: 2_BF    
ConfigGroup,Channel,2_BF,TIFilterBlock1,Label,2-BF

Preset: 3_DAPI   
ConfigGroup,Channel,3_DAPI,TIFilterBlock1,Label,3-DAPI

Preset: 4_eGFP   
ConfigGroup,Channel,4_eGFP,TIFilterBlock1,Label,4-GFP

Preset: 5_Cy5   
ConfigGroup,Channel,5_Cy5,TIFilterBlock1,Label,5-Cy5

Preset: 6_AttoPhos   
ConfigGroup,Channel,6_AttoPhos,TIFilterBlock1,Label,6-AttoPhos

In [ ]:
core.setConfig('Channel','1_PBP')

In [ ]:
core.snapImage()
img = core.getImage()
plt.imshow(img,cmap='gray')
image = Image.fromarray(img)
image.save('TESTIMAGE.tif')

In [ ]:
for i in range(5):
    x = core.getXPosition()
    y = core.getYPosition()
    core.setXYPosition(x-1500,y)
    core.waitForDevice(core.getXYStageDevice())
    
    core.snapImage()
    img = core.getImage()
    image = Image.fromarray(img)
    image.save('images/images_{}.tif'.format(i))

In [ ]:
core.unloadAllDevices()
core.reset()
print 'closed'

# Example: Get info

In [ ]:
core.getFocusDevice()
core.getCameraDevice()
core.XYStageDevice()
core.getDevicePropertyNames(core.getCameraDevice())

# Example: Shutter control

In [ ]:
# SHUTTER

# Auto
core.setAutoShutter(True)
core.snapImage()

# Manual
core.setAutoShutter(False) # disable auto shutter
core.setProperty("Shutter", "State", "1")
core.waitForDevice("Shutter")
core.snapImage()
core.setProperty("Shutter", "State", "0")

# Example: Video

In [ ]:
# cv2.startWindowThread()
cv2.namedWindow('Video')
cv2.imshow('Video',img)
cv2.waitKey(0)

In [ ]:
cv2.destroyAllWindows()
core.stopSequenceAcquisition()

In [ ]:
import cv2
cv2.namedWindow('Video')
core.startContinuousSequenceAcquisition(1)
while True:
    img = core.getLastImage()
    if core.getRemainingImageCount() > 0:
#         img = core.popNextImage()
        img = core.getLastImage()
        cv2.imshow('Video', img)
        cv2.waitkey(0)
    else:
        print('No frame')
    if cv2.waitKey(20) >= 0:
        break
cv2.destroyAllWindows()
core.stopSequenceAcquisition()
# core.reset()

# Example: Logging

In [ ]:
core.enableStderrLog(True)
core.enableDebugLog(True)

# Example: Config, programmatic

In [ ]:
## load devices
core.loadDevice("Camera", "DemoCamera", "DCam")
core.loadDevice("Emission", "DemoCamera", "DWheel")
core.loadDevice("Excitation", "DemoCamera", "DWheel")
core.loadDevice("Dichroic", "DemoCamera", "DWheel")
core.loadDevice("Objective", "DemoCamera", "DObjective")
core.loadDevice("X", "DemoCamera", "DStage")
core.loadDevice("Y", "DemoCamera", "DStage")
core.loadDevice("Z", "DemoCamera", "DStage")
core.initializeAllDevices()

## set labels for state devices
# emission filter
core.defineStateLabel("Emission", 0, "Chroma-D460")
core.defineStateLabel("Emission", 1, "Chroma-HQ620")
core.defineStateLabel("Emission", 2, "Chroma-HQ535")
core.defineStateLabel("Emission", 3, "Chroma-HQ700")
# excitation filter
core.defineStateLabel("Excitation", 2, "Chroma-D360")
core.defineStateLabel("Excitation", 3, "Chroma-HQ480")
core.defineStateLabel("Excitation", 4, "Chroma-HQ570")
core.defineStateLabel("Excitation", 5, "Chroma-HQ620")
# excitation dichroic
core.defineStateLabel("Dichroic", 0, "400DCLP")
core.defineStateLabel("Dichroic", 1, "Q505LP")
core.defineStateLabel("Dichroic", 2, "Q585LP")
# objective
core.defineStateLabel("Objective", 1, "Nikon 10X S Fluor")
core.defineStateLabel("Objective", 3, "Nikon 20X Plan Fluor ELWD")
core.defineStateLabel("Objective", 5, "Zeiss 4X Plan Apo")

## define configurations
core.defineConfiguration("FITC", "Emission", "State", "2")
core.defineConfiguration("FITC", "Excitation", "State", "3")
core.defineConfiguration("FITC", "Dichroic", "State", "1")
core.defineConfiguration("DAPI", "Emission", "State", "1")
core.defineConfiguration("DAPI", "Excitation", "State", "2")
core.defineConfiguration("DAPI", "Dichroic", "State", "0")
core.defineConfiguration("Rhodamine", "Emission", "State", "3")
core.defineConfiguration("Rhodamine", "Excitation", "State", "4")
core.defineConfiguration("Rhodamine", "Dichroic", "State", "2")

## set initial imaging mode
core.setProperty("Camera", "Exposure", "55")
core.setProperty("Objective", "Label", "Nikon 10X S Fluor")
core.setConfiguration("DAPI")

# Generated by Configurator on Mon Mar 20 15:37:38 PDT 2017

# Reset
Property,Core,Initialize,0

# Devices
Device,COM1,SerialManager,COM1
Device,COM6,SerialManager,COM6
Device,Cam Andor_Zyla4.2,AndorSDK3,Andor sCMOS Camera
Device,Scope Nikon_Ti,NikonTI,TIScope
Device,TIFilterBlock1,NikonTI,TIFilterBlock1
Device,XY ASI_MS2000,ASIStage,XYStage
Device,Z ASI_MS2000,ASIStage,ZStage
Device,Spectra,LumencorSpectra,Spectra

# Pre-init settings for devices
Property,COM1,AnswerTimeout,500.0000
Property,COM1,BaudRate,9600
Property,COM1,DelayBetweenCharsMs,0.0000
Property,COM1,Handshaking,Off
Property,COM1,Parity,None
Property,COM1,StopBits,1
Property,COM1,Verbose,1
Property,COM6,AnswerTimeout,500.0000
Property,COM6,BaudRate,9600
Property,COM6,DelayBetweenCharsMs,0.0000
Property,COM6,Handshaking,Off
Property,COM6,Parity,None
Property,COM6,StopBits,1
Property,COM6,Verbose,1
Property,XY ASI_MS2000,Port,COM1
Property,Z ASI_MS2000,Axis,Z
Property,Z ASI_MS2000,Port,COM1
Property,Spectra,Port,COM6
Property,Spectra,SetLE_Type,Sola

# Pre-init settings for COM ports

# Hub (parent) references
Parent,TIFilterBlock1,Scope Nikon_Ti

# Initialize
Property,Core,Initialize,1

# Delays
Delay,TIFilterBlock1,250.0

# Focus directions

# Roles
Property,Core,Camera,Cam Andor_Zyla4.2
Property,Core,Shutter,Spectra
Property,Core,Focus,Z ASI_MS2000
Property,Core,AutoShutter,1

# Camera-synchronized devices

# Labels
# TIFilterBlock1
Label,TIFilterBlock1,5,6-AttoPhos
Label,TIFilterBlock1,4,5-Cy5
Label,TIFilterBlock1,3,4-GFP
Label,TIFilterBlock1,2,3-DAPI
Label,TIFilterBlock1,1,2------
Label,TIFilterBlock1,0,1-BF

# Configuration presets
# Group: Channel
# Preset: 6-AttoPhos
ConfigGroup,Channel,6-AttoPhos,TIFilterBlock1,Label,6-AttoPhos

# Preset: 5-Cy5
ConfigGroup,Channel,5-Cy5,TIFilterBlock1,Label,5-Cy5

# Preset: 1------
ConfigGroup,Channel,1------,TIFilterBlock1,Label,1-BF

# Preset: 3-DAPI
ConfigGroup,Channel,3-DAPI,TIFilterBlock1,Label,3-DAPI

# Preset: 2------
ConfigGroup,Channel,2------,TIFilterBlock1,Label,2------

# Preset: 4-eGFP
ConfigGroup,Channel,4-eGFP,TIFilterBlock1,Label,4-GFP


# Group: SolaEnable
# Preset: NewPreset
ConfigGroup,SolaEnable,NewPreset,Spectra,White_Enable,1



# PixelSize settings
# Resolution preset: 10x
ConfigPixelSize,10x,Core,AutoShutter,1
PixelSize_um,10x,0.6418
ConfigPixelSize,10x,Core,Camera,Cam Andor_Zyla4.2
PixelSize_um,10x,0.6418
ConfigPixelSize,10x,Core,Focus,Z ASI_MS2000
PixelSize_um,10x,0.6418
ConfigPixelSize,10x,Core,Initialize,1
PixelSize_um,10x,0.6418
ConfigPixelSize,10x,Core,Shutter,Spectra
PixelSize_um,10x,0.6418
ConfigPixelSize,10x,Core,XYStage,XY ASI_MS2000
PixelSize_um,10x,0.6418


